In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import openpyxl
import re
import os
from variableUtils import *
import variableUtils
from Utils import *
from ClassUtils import *
from pprint import pprint
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image
from reportlab.lib import colors
from reportlab.platypus import Paragraph, Spacer, KeepTogether, KeepInFrame
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
import io
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule
import PIL

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(sns.__version__)
from datetime import datetime

# get today's date
today = datetime.now().strftime('%d-%m-%Y')
print(f"Today's date: {today}")


0.13.2
Today's date: 28-07-2025


## Research Design

In [48]:
workbookpath = '2025\Extra\Research Design.xlsx'
folder, file, ext = getFolderandFileName(workbookpath)
df = pd.read_excel(workbookpath)
# display(df.head())
# drop first row
df = df.drop(index=0)
display(df.head())
infoCols = [
    "Legal name",
    "Last name",
    "Preferred name",
    "Email",]

supervisorCols = [
    "Supervisors",
]
quesCols = [
#     "Supervisors",
    "Background",
    "Hypothetis/Aims/Objectives",
    "Research Methodology",
    "Conclusion",
    "Quality of writing",
    "Critical evaluation",
    "Scholarship",
]
totalCols = [    "Total",
    "Out of 20 points",
    "Percentage grade",]
colComments = "Comments"
maxScoreDict = {
    "Background": 5,
    "Hypothetis/Aims/Objectives": 5,
    "Research Methodology": 5,
    "Conclusion": 5,
    "Quality of writing": 5,
    "Critical evaluation": 5,
    "Scholarship": 5,
    "Total": 35,
    "Out of 20 points": 20,
    "Percentage grade": 100,
}
df['Out of 20 points'] = df['Out of 20 points'].round(2)
df['Percentage grade'] = df['Percentage grade'].round(2)

gradedescdf = pd.read_csv('2025\Extra\Rubric_DataFrame.csv')
# headers are grades and rows are criteria
gradedescdf = gradedescdf.set_index('Criteria')
display(gradedescdf.head())
print(gradedescdf.loc['Background', '5'])
savefolder = f'{folder}\\Research Design'
if not os.path.exists(savefolder):
    os.makedirs(savefolder)

,Unnamed: 0,Unnamed: 1,Legal name,Last name,Preferred name,Email,Supervisors,Background,Hypothetis/Aims/Objectives,Research Methodology,Conclusion,Quality of writing,Critical evaluation,Scholarship,Total,Out of 20 points,Percentage grade,Comments
1,Group 1,DDS2-Grp-01-A,Mr Aaron Ram,Ram,Aaron,asram@student.unimelb.edu.au,Bree Jones,3,4,5,4,4,3,3,26,14.857143,74.285714,Great overview of your research design and how...
2,NaN,DDS2-Grp-01-B,Ms Anjali Bhatnagar,Bhatnagar,Anjali,anjalib@student.unimelb.edu.au,Claire Mustchin,5,5,5,4,5,4,4,32,18.285714,91.428571,Very thorough background with clear descriptio...
3,NaN,DDS2-Grp-01-C,Mr Jonathan Gershon,Gershon,Jonathan,jgershon@student.unimelb.edu.au,Christopher Honig,3,3,5,3,4,2,3,23,13.142857,65.714286,"Good summary of generative AI, but less detail..."
4,NaN,DDS2-Grp-01-D,Ms Chuhan Dong,Dong,Bella,chuhand@student.unimelb.edu.au,NaN,5,5,5,5,5,4,4,33,18.857143,94.285714,Very clear and thorough background highlightin...
5,NaN,DDS2-Grp-01-E,Rita Lan,Lan,Rita,rllan@student.unimelb.edu.au,NaN,5,5,5,5,5,4,4,33,18.857143,94.285714,Fantastic summary of the background for this r...


,5,4,3,2,0
Criteria,,,,,
Background,"Background is succinct, well-structured and lo...","Background is structured, logical and gives in...",Background is appropriate and adequate but not...,Background appears disjointed but somehow rela...,Background is not relevant and does not lead t...
Hypothetis/Aims/Objectives,"The hypothesis, aims and/or objectives are cle...",The hypothesis aims and/or objectives are clea...,"The hypothesis, aims and/or objectives suggest...",The hypothesis aims and/or objectives are eith...,The hypothesis aims and/or objectives are miss...
Quality of writing,All writing is of an excellent scientific and ...,Writing is of a high scientific and profession...,Writing is of a solid scientific and professio...,Writing meets basic scientific and professiona...,Writing falls below acceptable scientific and ...
Critical evaluation,Demonstrates insight into flaws/limitations of...,Demonstrates insight into flaws/limitations of...,Some critical evaluation included but not part...,Critical evaluation of the literature is brief...,Critical evaluation of the literature is lacki...
Research Methodology,"The methodology was clearly presented, it can ...","The methodology was presented, it can be easil...","The methodology was adequate, it can be reprod...","The methodology was problematic, difficult to ...",The methodology was poor and difficult to foll...


Background is succinct, well-structured and logical and gives insight into why the study is necessary. It leads the reader to ask the same research questions.


In [35]:
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.enums import TA_CENTER
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

# Register Arial if available
pdfmetrics.registerFont(TTFont('Arial', 'arial.ttf'))

bannerHeadingStyle = ParagraphStyle(
    name='BannerHeading',
    fontName='Helvetica-Bold',  # ensures actual bold
    fontSize=18,
    textColor='white',
    alignment=0,
    leading=22,
    spaceAfter=0,
    spaceBefore=0
)
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

pdfmetrics.registerFont(TTFont('Calibri-Bold', 'calibrib.ttf'))  # Bold version



def getBannerDrawer(name):
    def drawBanner(canvas, doc):
        canvas.saveState()

        # Banner layout
        pageWidth, pageHeight = doc.pagesize
        bannerHeight = 132
        canvas.setFillColor(colors.HexColor("#010d44"))
        canvas.rect(0, pageHeight - bannerHeight, pageWidth, bannerHeight, fill=1, stroke=0)

        # Text: internal margin from left and top
        textLeftMargin = leftMargin
        topOffset = 72  # Distance from top of banner to first text line
        lineSpacing = 36

        try:
            canvas.setFont("Calibri-Bold", 30)
        except:
            canvas.setFont("Helvetica-Bold", 30)  # Fallback

        canvas.setFillColor(colors.white)
        canvas.drawString(textLeftMargin, pageHeight - topOffset, "Research Design Feedback")
        try:
            canvas.setFont("Calibri-Bold", 24)
        except:
            canvas.setFont("Helvetica-Bold", 24)  # Fallback        
        
        canvas.drawString(textLeftMargin, pageHeight - topOffset - lineSpacing, f"{name}")

        canvas.restoreState()

    return drawBanner

In [ ]:
def createReport(row):
    name = row['Preferred name'] + ' ' + row['Last name']

    doc = SimpleDocTemplate(f'{savefolder}\\{name}.pdf', pagesize=pageSize, leftMargin=leftMargin,
                                rightMargin=rightMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    elements = []
    # elements.append(Paragraph(f'Research Design Report for {name}', headingStyle))
    # elements.append(Spacer(0, 0.2 * inch))
    elements.append(Spacer(1, 72))
    for col in infoCols + supervisorCols + [colComments]:
        val = row[col]
        if isinstance(val, list):
            if not val:
                continue  # skip if empty list
            val = ','.join(val).strip(',')  # convert list to string
        elif pd.isna(val) or str(val).strip() == 'nan':
            continue
        print(f'{col}: {row[col]}')
        elements.append(Paragraph(f'{col.strip()}', smallsubsubheadingStyleL))
        elements.append(Spacer(1, 6))
        text = val.strip().replace('\n', '<br/>')
        elements.append(Paragraph(f'{text}', tableTextStyleLSmall))
    
    # create a table for questions
    elements.append(Spacer(1, 12))
    tableDf= pd.DataFrame(columns = ['Criteria', 'Grade', 'Grade Description'])
    for col in quesCols:
        if col not in row or pd.isna(row[col]) or str(row[col]).strip() == 'nan':
            continue
        grade = row[col]
        if isinstance(grade, list):
            grade = ','.join(grade).strip(',')
        elif isinstance(grade, str):
            grade = grade.strip()
        if grade == '':
            continue
        tableDf = pd.concat([tableDf, pd.DataFrame({
            'Criteria': [f'{col}'],
            'Grade': [f'{grade}/{maxScoreDict[col]}'],
            'Grade Description': [f'{gradedescdf.loc[col, str(grade)] if col in gradedescdf.index else "No description available"}']
        })], ignore_index=True)

    if not tableDf.empty:
        table = createTable(tableDf, colRatio = [2, 1, 5], customTextCols=[0, 1, 2], tableTextStyle=tableTextStyleSmall, topPadding= 6, bottomPadding= 6, title= '',
                            headerColor='#bebebe', headerTextColor='#000000', titleStyle=subsubheadingStyle)
        elements.append(table)

    # add total scors
    elements.append(Spacer(1, 12))
    for col in totalCols:
        if col not in row or pd.isna(row[col]) or str(row[col]).strip() == 'nan':
            continue
        val = row[col]
        if isinstance(val, list):
            val = ','.join(val).strip(',')
        elif isinstance(val, str):
            val = val.strip()
        if col == 'Percentage grade':
            val = f'{val:.2f}%'  # format as percentage
            elements.append(Paragraph(f'{col.strip()}: {val}', smallsubsubheadingStyleL))
        else:
            elements.append(Paragraph(f'{col.strip()}: {val}/{maxScoreDict[col]}', smallsubsubheadingStyleL))
        elements.append(Spacer(1, 6))


    doc.build(elements, onFirstPage=getBannerDrawer(name))
for name in df['Legal name'].unique():
    print(f'Creating report for {name}')
    # skip if name is empty
    if pd.isna(name) or str(name).strip() == '':
        print(f'Skipping empty name')
        continue
    row = df[df['Legal name'] == name].iloc[0]
    createReport(row)
    # break


Creating report for Mr Aaron Ram
Legal name: Mr Aaron Ram
Last name: Ram
Preferred name: Aaron
Email: asram@student.unimelb.edu.au
Supervisors: Bree Jones
Comments: Great overview of your research design and how this would contribute to the field. What are the benefits of using the AI chatbox for patient care? For future assignments, consider any limitations associated with this study, is using 1 population (MDS) providing an outcome that would applicable to other dental schools? Inclusion of some more critical evalutation on the current practices and the potential of AI chatbots would be beneficial. I would avoid bullet points in areas of main text, as opposed to the research methodology or objectives. 
Creating table for 
Creating report for Ms Anjali Bhatnagar
Legal name: Ms Anjali Bhatnagar
Last name: Bhatnagar
Preferred name: Anjali
Email: anjalib@student.unimelb.edu.au
Supervisors: Claire Mustchin 
Comments: Very thorough background with clear description of why this research is 

In [51]:
# send emails with reports\
import win32com.client
import win32com
outlook = win32com.client.Dispatch("Outlook.Application")
# get full path of savefolder
savefolder = os.path.abspath(savefolder)
for i, row in df.iterrows():
    if pd.isna(row['Legal name']) or str(row['Legal name']).strip() == '':
        print(f'Skipping row {i} due to missing Legal name')
        continue
    name = row['Preferred name'] + ' ' + row['Last name']
    email = row['Email']
    if pd.isna(email) or str(email).strip() == '':
        print(f'Skipping {name} due to missing email')
        continue
    subject = f'Research Design Feedback for {name}'
    body = f'Dear {name},\n\nPlease find attached your Research Design Feedback report.\n\nBest regards,'
    attachment = f'{savefolder}\\{name}.pdf'
    print(f'Sending email to {name} at {email} with attachment {attachment}')
    try:
        mail = outlook.CreateItem(0)
        mail.Subject = subject
        mail.Body = body
        mail.To = email
        mail.Attachments.Add(attachment)
        mail.Send()
        print(f'Email sent to {name} at {email}')
    except Exception as e:
        print(f'Failed to send email to {name} at {email}: {e}')
    # break  # uncomment to send only one email for testing

Sending email to Aaron Ram at asram@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS Work\2025\Extra\Research Design\Aaron Ram.pdf
Email sent to Aaron Ram at asram@student.unimelb.edu.au
Sending email to Anjali Bhatnagar at anjalib@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS Work\2025\Extra\Research Design\Anjali Bhatnagar.pdf
Email sent to Anjali Bhatnagar at anjalib@student.unimelb.edu.au
Sending email to Jonathan Gershon at jgershon@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS Work\2025\Extra\Research Design\Jonathan Gershon.pdf
Email sent to Jonathan Gershon at jgershon@student.unimelb.edu.au
Sending email to Bella Dong at chuhand@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D folder\MDS Work\2025\Extra\Research Design\Bella Dong.pdf
Email sent to Bella Dong at chuhand@student.unimelb.edu.au
Sending email to Rita Lan at rllan@student.unimelb.edu.au with attachment c:\Users\Kunal Patel\D 

## Written Exams

In [49]:
workbookpath = '2025\Extra\Written Exams\Exam_ Oral Health Practice 2 (ORAL20005_2025_YRL)- Requires Respondus LockDown Browser Quiz Student Analysis Report.csv'
workbookpath = '2025\Extra\Written Exams\Exam_ Oral Health Sciences 2 (ORAL20003_2025_YRL) Supplementary - Requires Respondus LockDown Browser Quiz Student Analysis Report.xlsx'
workbookpath = '2025\Extra\Written Exams\Exam_ Oral Health Sciences 2 (ORAL20003_2025_YRL)- Requires Respondus LockDown Browser Quiz Student Analysis Report 2.xlsx'
folder, file, ext = getFolderandFileName(workbookpath)
for enc in ['utf-8', 'ISO-8859-1', 'utf-16']:
    try:
        df = pd.read_csv(workbookpath, encoding=enc) if workbookpath.endswith('.csv') else pd.read_excel(workbookpath)
        print(f"✅ Loaded using {enc}")
        break
    except Exception as e:
        print(f"❌ Failed with {enc}: {e}")
display(df.head())
question_columns = [(col, df[col]) for col in df.columns if isinstance(col, str) and ':' in col]
score_cols = ['n correct', 'n incorrect', 'score']

✅ Loaded using utf-8


,name,id,sis_id,section,section_id,section_sis_id,submitted,attempt,2530308: Which of the following factors most significantly increases the risk of developing medication-related osteonecrosis of the jaw (MRONJ)?,1,2530309: What is the correct statement regarding the relationship between diabetes and periodontal disease?,1.1,2530310: Which of the following types of pain can result because of damage to the nervous system?â€¯,1.2,2530311: A middle-aged Australian man who lives in community housing has Stage 3 grade B periodontitis and requires a sub-gingival scaling. They have no relevant medical history and no listed medications.\nPlease select the appropriate consideration for antibiotic prophylaxis.\n,1.3,2530312: Which of the following contribute to gingival recession?,1.4,2530314: Tertiary dentine formation is a critical response from the dentine pulp complex when challenged by caries. Which of the following statements about tertiary dentine isâ€¯TRUE?,1.5,"2530316: Intermittent, sharp facial pain described as an electric shock after light touch, unilateral in presentation without obvious signs of periodontal infection or caries is best described as:\n",1.6,2530317: Which of the following statements about caries progression into dentine isâ€¯CORRECT?â€¯â€¯,1.7,2530318: Select theâ€¯CORRECTâ€¯statement regarding calcium and phosphate in the demineralisation process:\n,1.8,2530319: Which the following statements best describes the principles of antimicrobial stewardship:,1.9,2530320: The most effectiveâ€¯bufferingâ€¯of an intraoral acid challenge is from:,1.10,"2530321: A patient reports pain that is worse upon waking in the morning. They describe the pain as being sore on the right-hand side and the left-hand side, and the pain jumps around. The pain has tender muscles of mastication.\nBased on this pain history, the pain described is more likely to be of which origin?",1.11,2530322: Which of the following would be most useful for helping you discriminate a periodontal abscess from a periapical abscess?,1.12,2530323: Systemic factors are important in the pathogenesis of periodontal disease because:,1.13,"2530324: In an acidic pH (in the case of an abscess), what happens to the ionisation of LA as it is a weak base?",1.14,2530326: Your patient attends for a large occlusal restoration on tooth 35.â€¯\nWhich of the following anaesthetic techniques will you use to anaesthetise the pulp?,1.15,2530327: Which of the following is an absolute contraindication for using an adrenaline containing lignocaine anaesthetic solution?,1.16,2530330: Which of the following is the MOST correct statement about the differences between caries and erosion?,1.17,2530335: You are administering lignocaine 2% with 1:80 000 adrenaline (2.2ml) for an Inferior Alveolar Nerve block to perform local anaesthesia for a pulpotomy procedure of the 75. The child is 8 years old and weighs 27kgs.â€¯\nCalculate the maximum recommended dose that can be administered for this patient and state the maximum number of cartridges this equates to.\nUse the below box from the Therapeutic Guidelines to aid in your calculations.\n,2,2542259: A 42-year-old patient attends her appointment with you and has recently been diagnosed with temporomandibular disorder.\nThere is one part to this question. Please respond in the box below.â€¯\n\n How is this diagnosis defined and how is it different from odontogenic pain?â€¯ (10 Marks)â€¯\n,10,2542260: There are two parts to this question. Please respond in the box below and label each A and B.\nA. After completion of a Right-Hand Side Inferior Alveolar Nerve block you observe the patient has unilateral facial paralysis on the same side as injection.â€¯â€¯\nOutline why this has occurred and the most important consideration in managing this. (6 Marks)\nB. Cindy is 5 and needs and extraction of the 54. She has buccal facial swelling.\nWould this Impact the onset and effectiveness of local anaesthesia at this site. why/why not? (4 marks),10.

In [50]:
from reportlab.platypus import Flowable

class EditableCommentField(Flowable):
    def __init__(self, name, value, label='', width=600, height=180):
        super().__init__()
        self.name = name
        self.value = value or ""
        self.label = label
        self.width = width
        self.height = height
        self.fieldHeight = height + 16  # extra space for label

    def wrap(self, availWidth, availHeight):
        return availWidth, self.fieldHeight

    def draw(self):
        c = self.canv
        # Get absolute position of (0,0) in this Flowable
        x, y = c.absolutePosition(0, 0)


        # Draw label just above the field
        # c.drawString(x, y + self.height + 2, f"{self.label}:")

        # Draw editable field using absolute positioning
        c.acroForm.textfield(
            name=self.name,
            tooltip=self.label,
            x=x, y=y,
            width=self.width,
            height=self.height,
            value=self.value,
            borderStyle='underlined',
            forceBorder=True,
            fieldFlags=4096,  # Enables multiline
            # fillcolor = colors.HexColor("#bebebe"),  # Light grey background
        )
        # c.setFillColor(colors.HexColor("#bebebe"))  # Light grey
def sanitizeTextForPDF(text):
    if not text:
        return ''
    return (
        text.replace("’", "'")   # RIGHT SINGLE QUOTE to ASCII apostrophe
            .replace("‘", "'")   # LEFT SINGLE QUOTE
            .replace("“", '"')   # LEFT DOUBLE QUOTE
            .replace("”", '"')   # RIGHT DOUBLE QUOTE
            .replace("–", "-")   # EN DASH
            .replace("—", "-")   # EM DASH
            .replace("…", "...") # Ellipsis
            .encode('ascii', 'ignore')  # strip out other weird stuff
            .decode('ascii')
    )
def cleanText(text):
    # Remove non-printable and non-ASCII characters
    return re.sub(r'[^\x20-\x7E]+', '', text)

In [51]:

for idx, row in df.iterrows():
    # if idx >= 2:
    #     break

    student_name = row['name']
    student_id = row['sis_id']
    savefolder = '2025/Extra/Written Exams/Reports 2'
    os.makedirs(savefolder, exist_ok=True)
    file_name = f"{savefolder}/{student_name} ({student_id}).pdf"

    doc = SimpleDocTemplate(f'{file_name}', pagesize=pageSize, rightMargin = rightMargin, 
                            leftMargin = leftMargin, topMargin = topMargin, bottomMargin = bottomMargin)
    elements = []

    # Title and student info
    elements.append(Paragraph("Written Exam Report – ORAL20005", headingStyle))
    elements.append(Spacer(1, 0.2 * inch))
    elements.append(Paragraph("Section: Oral Health Practice 2 (ORAL20005_2025_YRL)", subheadingStyle))
    elements.append(Spacer(1, 0.1 * inch))
    elements.append(Paragraph(f"Student: {student_name} | ID: {student_id}", subsubheadingStyle))
    elements.append(Spacer(1, 0.3 * inch))

    # Questions
    for i, (question_text, _) in enumerate(question_columns, 1):
        grade_col = df.columns[df.columns.get_loc(question_text) + 1]
        print(grade_col)
        maxGrade = str(grade_col).split('.')[0]
        maxGrade = int(maxGrade)
        answer_text = str(row[question_text])
        answer_text = answer_text.replace("\n", "<br/>")
        question_text = question_text.split(": ", 1)[-1]
        # clean the answer text and question text
        question_text = cleanText(question_text)
        answer_text = cleanText(answer_text)
        print(f"Processing Question {i}: {question_text} with answer: {answer_text}")
        if pd.isna(answer_text) or str(answer_text).strip() == '':
            continue
        student_score = row[grade_col]
        avg_score = df[grade_col].mean()

        # elements.append(Paragraph(f"Question {i}", smallsubsubheadingStyleL))
        elements.append(Paragraph(f"Question {i}: " + question_text, smallsubsubheadingStyleL))
        elements.append(Paragraph(f"Student Answer: {sanitizeTextForPDF(answer_text)}", tableTextStyleLSmall))
        elements.append(Paragraph(f"Student Grade: {int(student_score)}/{maxGrade}", smallsubsubheadingStyleL))
        elements.append(Paragraph(f"Class Average Grade: {avg_score:.2f}", smallsubsubheadingStyleL))
        elements.append(Spacer(1, 0.1 * inch))
        elements.append(Paragraph("Space for comments about this question:", smallsubsubheadingStyleL))
        elements.append(EditableCommentField(
                name=None,
                value="",
                label="Space for comments about this question",
                width = 750,
                height= 80,
            ))
        elements.append(Spacer(1, 0.1 * inch))
        elements.append(Paragraph("Students who answered this question well did this:", smallsubsubheadingStyleL))
        elements.append(EditableCommentField(
                name=None,
                value="",
                label="Students who answered this question well did this…",
                width = 750,
                height= 80,
            ))
        elements.append(Spacer(1, 0.1 * inch))
        elements.append(Paragraph("Areas to improve in this question are:", smallsubsubheadingStyleL))
        elements.append(EditableCommentField(
                name=None,
                value="",
                label="Areas to improve in this question are",
                width = 750,
                height= 80,
            ))
        if i%1 == 0:
            elements.append(PageBreak())
        else:
            elements.append(Spacer(1, 0.3 * inch))

    # Add scores
    elements.append(Spacer(1, 0.2 * inch))
    for col in score_cols:
        if col not in row or pd.isna(row[col]) or str(row[col]).strip() == 'nan':
            continue
        val = row[col]
        if isinstance(val, list):
            val = ','.join(val).strip(',')
        elif isinstance(val, str):
            val = val.strip()
        if col == 'score':
            val = f'{val:.2f}%'  # format as percentage
            elements.append(Paragraph(f'{col.strip()}: {val}', subsubheadingStyle))
        else:
            elements.append(Paragraph(f'{col.strip()}: {val}', subsubheadingStyle))
        elements.append(Spacer(1, 6))

    doc.build(elements)

1
Processing Question 1: Which of the following factors most significantly increases the risk of developing medication-related osteonecrosis of the jaw (MRONJ)? with answer: Long-term bisphosphonate use
1.1
Processing Question 2: What is the correct statement regarding the relationship between diabetes and periodontal disease? with answer: Increased response to inflammation, poor wound healing, delayed healing response
1.2
Processing Question 3: Which of the following types of pain can result because of damage to the nervous system? with answer: Neuropathic pain
1.3
Processing Question 4: A middle-aged Australian man who lives in community housing has Stage 3 grade B periodontitis and requires a sub-gingival scaling. They have no relevant medical history and no listed medications.Please select the appropriate consideration for antibiotic prophylaxis. with answer: No prophylaxis is required
1.4
Processing Question 5: Which of the following contribute to gingival recession? with answer: 

ValueError: cannot convert float NaN to integer